# 建立sample:name字典

In [1]:
def getnamedict(txt='sample_name.txt'):
    with open(txt,'r') as f:
        name = f.read().split("\n")
        name_dict = {}
        for i in name:
            key, value = i.split(":")
            name_dict[key] = value
    return name_dict

In [2]:
name_dict = getnamedict(txt='sample_name.txt')
number_of_samples = len(name_dict)

# 載入訓練好的模型

In [3]:
from keras.models import load_model
classifier = load_model('SaveModel/face.hd5')

Using TensorFlow backend.


# 人臉辨識

In [4]:
import numpy as np          #Python進行科學計算的基礎包
import dlib                 #dlib是一套包含了機器學習、計算機視覺、圖像處理等的函式庫
import cv2                  #OpenCV

def facerecognition(film):  #人臉辨識的函數，參數為(VideoCapture參數)
    cap = cv2.VideoCapture(film)                                #開啟影片檔案
    detector = dlib.get_frontal_face_detector()              #Dlib的人臉偵測器

    while(cap.isOpened()):       #使用cap.isOpened()，來檢查是否成功初始化，以迴圈從影片檔案讀取影格，並顯示出來
        ret, frame = cap.read()  #第一個參數ret的值為True或False，代表有沒有讀到圖片;第二個參數是frame，是當前截取一幀的圖片。
        face_rects, scores, idx = detector.run(frame, 0)     #偵測人臉
        for i, d in enumerate(face_rects):                   #取出所有偵測的結果
            x1 = d.left()
            y1 = d.top()
            x2 = d.right()
            y2 = d.bottom()
            cropped = frame[int(y1):int(y2),int(x1):int(x2)] #裁剪偵測到的人臉
            image=cv2.resize(cropped,(64, 64),interpolation=cv2.INTER_CUBIC) #將人臉圖片大小調整為(64, 64)
            image = np.expand_dims(image, axis = 0)          #增加一個維度
            label = str(classifier.predict_classes(image)[0])#將預測類別的型態轉為字串
            label = name_dict['sample'+label]                #利用字典找姓名
            text = label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA) #以方框標示偵測的人臉，cv2.LINE_AA為反鋸齒效果
            cv2.putText(frame, text, (x1, y1), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)  #標示姓名
        cv2.imshow("face recognition", frame)                  #顯示結果
        if cv2.waitKey(1) & 0xFF == ord('q'):                #按Q停止
            break

    cap.release()                                            #釋放資源
    cv2.destroyAllWindows()                                  #刪除任何我們建立的窗口

In [5]:
facerecognition(film=0)  #人臉辨識的函數，參數為(VideoCapture參數)